<a href="https://colab.research.google.com/github/abxda/python/blob/main/Spark_Kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [ ]:
!apt install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
#Basado en https://colab.research.google.com/drive/11RzVuPw01cOGj9Z4B_VV681ZyXKCSkQN?usp=sharing#scrollTo=A_XknnI91JCW

# required to change logging level, otherwise we could have got away with pip3 install
#
!wget -q https://dlcdn.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz
!tar xf spark-3.4.3-bin-hadoop3.tgz


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

!pip3 install -q pyspark

!cp $SPARK_HOME/conf/log4j2.properties.template $SPARK_HOME/conf/log4j2.properties                  # copy the template
!sed -i 's/rootLogger.level = info/rootLogger.level = warn/g' $SPARK_HOME/conf/log4j2.properties    # make the change
!diff $SPARK_HOME/conf/log4j2.properties.template $SPARK_HOME/conf/log4j2.properties

In [ ]:
!wget -q https://downloads.apache.org/kafka/3.6.2/kafka_2.12-3.6.2.tgz
!tar xf kafka_2.12-3.6.2.tgz

In [ ]:
#Ejecutar en la terminal
#cd kafka_2.12-3.6.2
#bin/zookeeper-server-start.sh config/zookeeper.properties > /dev/null 2>&1 &
#sleep 10
#bin/kafka-server-start.sh config/server.properties > /dev/null 2>&1 &
#sleep 10
#bin/kafka-topics.sh --create --topic promedios --bootstrap-server localhost:9092
#bin/kafka-console-producer.sh --topic promedios --bootstrap-server localhost:9092

In [ ]:
%xterm

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession. \
builder. \
appName('UNIR'). \
config("spark.executor.memory", "5g"). \
config("spark.driver.memory", "10g"). \
config('spark.dirver.maxResultSize', '1g'). \
config('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.3'). \
getOrCreate()

In [ ]:
tiposStreamingDF = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .option("subscribe", "promedios")
  .load())


In [ ]:
tiposStreamingDF.printSchema()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import pyspark.sql.functions as F
esquema = StructType([\
  StructField("tipo", StringType()),\
  StructField("total", DoubleType())\
])

In [ ]:

parsedDF = tiposStreamingDF\
     .select("value")\
     .withColumn("value", F.col("value").cast(StringType()))\
     .withColumn("parejas", F.from_json(F.col("value"), esquema))\
     .withColumn("tipo", F.col("parejas.tipo"))\
     .withColumn("total", F.col("parejas.total"))


In [ ]:
parsedDF.printSchema()

In [ ]:
promediosStreamingDF = parsedDF \
    .groupBy("tipo") \
    .agg(F.avg("total").alias("promedio")) \
    .orderBy(F.desc("promedio"))

In [ ]:
salida = promediosStreamingDF\
                    .writeStream\
                    .queryName("AgregacionPromedios")\
                    .outputMode("complete")\
                    .format("memory")\
                    .start()

In [ ]:
promediosDF = spark.table("AgregacionPromedios")
promediosDF.show()

In [ ]:
{"tipo": "gasto", "total": 1.5}
{"tipo": "gasto", "total": 2.5}

In [ ]:
promediosDF.show()

In [ ]:
{"tipo": "ingreso", "total": 7.0}

In [ ]:
promediosDF.show()

In [ ]:
{"tipo": "ingreso", "total": 6.5}

In [ ]:
promediosDF.show()

In [ ]:
salida.stop()